In [36]:
### Task 1 - import files

import pandas as pd

path = 'datasets/'

class_csv = pd.read_csv(path + 'class.csv', sep=';')
test_level = pd.read_csv(path + 'test_level.csv', sep=';')
test = pd.read_csv(path + 'test.csv', sep=';')

In [37]:
### Task 2 - check files correctness 

# Select only rows with integer id
test = test[test['student_id'].apply(lambda x: str(x).isdigit())]
test = test[test['class_id'].apply(lambda x: str(x).isdigit())]
test = test[test['test_level_id'].apply(lambda x: str(x).isdigit())]
test = test[test['licence_id'].apply(lambda x: str(x).isdigit())]

# Clear empty overall_score and authorized_at rows
test = test[test['test_status'].notna()]
test = test[test['authorized_at'].notna()]

In [38]:
### Task 3 - prepare test_utilization.csv

import numpy as np
import datetime

# Transform date ie. 06.07.18 11:37 to 2018-07-06 (YY-MM-DD format)
def adjust_format_to_date(row):
    dd_c = datetime.datetime.strptime(row['test_created_at'][:8],'%d.%m.%y')
    dd_a = datetime.datetime.strptime(row['test_authorized_at'][:8],'%d.%m.%y')
    dd_c = str(dd_c.year) + '-' + '%02d' % dd_c.month + '-' + '%02d' % dd_c.day
    dd_a = str(dd_a.year) + '-' + '%02d' % dd_a.month + '-' + '%02d' % dd_a.day
    return dd_c, dd_a

# Get 'name' and 'teaching hours' from class_csv
def get_name(row):
    desired_row = class_csv[class_csv['id'] == row['class_id']]
    return desired_row.values[0][3], desired_row.values[0][6]
    
# Copy from test desired columns in test_utilization.csv and sort by 'class_id'
df = test.sort_values(by = ['class_id'])
df = df[['class_id','created_at','authorized_at','test_level_id']]
df.columns = ['class_id', 'test_created_at','test_authorized_at','test_level']

# Change 'test_created_at' and 'test_authorized_at' date format
df[['test_created_at', 'test_authorized_at']] = df.apply(adjust_format_to_date, axis=1, result_type="expand")

# Take class 'name' and 'teaching hours' from class_csv
df[['class_name', 'teaching_hours']] = df.apply(get_name, axis=1, result_type="expand")

# Enumerate test_id from 1 to the range of data frame
df['test_id'] = np.arange(1, df.shape[0]+1)

# Make class_test_number
df['class_test_number'] = df.groupby('class_id').cumcount()+1

# Sort columns to desired order
test_utilization = df[['class_id', 'class_name', 'teaching_hours', 'test_id', 'test_created_at', 'test_authorized_at', 'test_level', 'class_test_number']]

In [39]:
### Task 4 - prepare test_average_scores.csv

# Take idx of test marked as SCORING_SCORED
df = test[test['test_status'] == 'SCORING_SCORED']
df = df[['class_id', 'created_at', 'authorized_at', 'overall_score']]
df.columns = ['class_id', 'test_created_at', 'test_authorized_at', 'overall_score']

# Change 'test_created_at' and 'test_authorized_at' date format
df[['test_created_at', 'test_authorized_at']] = df.apply(adjust_format_to_date, axis=1, result_type="expand")

# Take the earliest/latest date from particular class (creation date/final authorization date)
min_date = df.groupby(['class_id']).agg({'test_created_at': [np.min]})
max_date = df.groupby(['class_id']).agg({'test_authorized_at': [np.max]})

# Calculate avg_class_test_overall_score
avg_score = df[['class_id','overall_score']].groupby('class_id').mean().round(2)

# Join avg_class_test_overall_score with min_date and max_date 
avg_score = avg_score.join(min_date).join(max_date)
avg_score = avg_score.reset_index()
avg_score.columns = ['class_id', 'avg_class_test_overall_score', 'test_created_at', 'test_authorized_at']

# Take class 'name' and 'teaching hours' from class_csv
avg_score[['class_name', 'teaching_hours']] = avg_score.apply(get_name, axis=1, result_type="expand")

# Sort columns to desired order
avg_score = avg_score[['class_id', 'class_name', 'teaching_hours', 'test_created_at', 'test_authorized_at', 'avg_class_test_overall_score']]

/opt/miniconda3/envs/mgr/lib/python3.7/site-packages/pandas/core/reshape/merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [40]:
### Task 5 - Save dataframes to the csv

test_utilization.to_csv(path + 'test_utilization.csv')
avg_score.to_csv(path + 'test_average_scores.csv')

In [41]:
### Task 6 - Save datasets to DB tables

import pandas as pd 
import sqlite3

# Read csv files
df_avg_score = pd.read_csv(path + 'test_average_scores.csv')
df_test_util = pd.read_csv(path + 'test_utilization.csv')

# Drop redundant idx column
df_avg_score = df_avg_score.loc[:, df_avg_score.columns != 'Unnamed: 0']
df_test_util = df_test_util.loc[:, df_test_util.columns != 'Unnamed: 0']

# Take column names 
columns_avg = str(tuple(df_avg_score.columns.values))
columns_util = str(tuple(df_test_util.columns.values))

# Make database connection
con = sqlite3.connect(":memory:") 
cur = con.cursor()

# Create dedicated tables - each for one csv.
cur.execute("CREATE TABLE avg_score " + columns_avg + ";") 
cur.execute("CREATE TABLE columns_util " + columns_util + ";") 

# Insert data into sqlite database
df_avg_score.to_sql('avg_score', con, if_exists='append', index=False)
df_test_util.to_sql('columns_util', con, if_exists='append', index=False)

# See db result - unconment below 2 lines 
print(pd.read_sql_query("SELECT * FROM avg_score", con))
print(pd.read_sql_query("SELECT * FROM columns_util", con))

     class_id            class_name teaching_hours test_created_at  \
0           1             QA Sanity           6-10      2018-07-06   
1           5  new tell app release            3-5      2018-07-13   
2           6               0.6.0.0            1-3      2018-07-18   
3           8                DCtest          11-15      2018-07-26   
4          10               0.7.0.0            3-5      2018-08-02   
..        ...                   ...            ...             ...   
438       647                   7-B            3-5      2019-04-23   
439       648                   7-C            3-5      2019-04-23   
440       649                   7-D            3-5      2019-04-23   
441       650                   7-E            3-5      2019-04-23   
442       652        ID-ALI-STEP UP            1-3      2019-04-25   

    test_authorized_at  avg_class_test_overall_score  
0           2018-07-06                         16.50  
1           2018-07-13                         17

In [42]:
### Task 7 - unit testing

from pandas.util.testing import assert_frame_equal, assert_series_equal, assert_index_equal
from task2 import check_files_correctness 
import pandas as pd
import unittest

class DFTests(unittest.TestCase):

    def setUp(self, filename, path):
        """ Check set up of the csv """
        try:
            data = pd.read_csv(path + filename, sep = ';')
            
        except IOError:
            print ('Cannot open the file')
            
        self.fixture = data

    def check_NaN_filtering(self):
        """ Check if function in task2 clear the dataset in a proper way """        
        empty_df = pd.DataFrame([],columns = ['id', 'student_id', 'class_id',  'created_at',  'updated_at', 'last_event_time',  'overall_score', 'test_status',	'institution_id', 'authorized_at', 'confidence_level',  'speaking_score', 'writing_score',  'reading_score',  'listening_score',  'test_level_id',  'licence_id'])

        # Test few cases of data filtering from task2  
        case1 = check_files_correctness(self.fixture.iloc[2:3,])
        case2 = check_files_correctness(self.fixture.iloc[3:4,:])
        case3 = check_files_correctness(self.fixture.iloc[37:38,])
        case4 = check_files_correctness(self.fixture.iloc[41:42,])
    
        # If left != right rise an error
        assert_frame_equal(case3, self.fixture.iloc[37:38,])
        #assert_frame_equal(case3, self.fixture.iloc[37:38,])
        #assert_frame_equal(case3, self.fixture.iloc[37:38,])
        #assert_frame_equal(case3, self.fixture.iloc[37:38,])
        
if __name__ == '__main__':
    # Filename which opening will be tested
    filename, path =  'test.csv', 'datasets/'

    # Test part 
    DFTests.setUp(unittest, filename, path)
    DFTests.check_NaN_filtering(unittest)

    print("Test finished!")


Assert error [8, 23, 'Class 1A', '15+', '2018-08-19', '2018-12-20', 18.4]
